![](https://ars.els-cdn.com/content/image/1-s2.0-S1574954119300895-gr1_lrg.jpg)

# Overview

With the effects of climate change impacting the globe now more than ever, it is critical to maintain healthy natural areas as much as possible. Human activity has a large impact on environmental health, and tracking that impact is critical. Activities like logging, mining, and agriculture can destroy the functioning of natural areas.  In each ecosystem, there are plants and animals that have an outsized impact on the healthy balance found in the region. These keystone species are critical to the survival of many other species in the region. Performing an accurate account of the species present can give insight into the health of a given area, and can help pinpoint areas that are in special need of immediate restoration efforts. 

# Business Understanding

The Mexican Government has partnered with the VIGIA project, which aims to create an autonomous surveillance system for monitoring the ecological health of protected natural areas. The first step in this project is to use drone footage to identify critical, or keystone species in the images. The initial images are from the Tehuacán-Cuicatlán Valley, a semi-arid zone in southern Mexico. This area was chosen because of its wide biodiversity and the importance that the ecology plays in the health of the region. It became a UNESCO World Heritage Site in 2018 in an effort to protect the area. Despite this protection, human activity is still impacting the health of the region. 

Our work focuses on columnar cactus recognition as they are a keystone species in the region. Our dataset includes over 16,000 RGB images representing an area of over 10,000 km^2. The images were taken by a DJI Phantom 3 Advanced drone flying at 100m. The images were manually identified and marked. 

Care must be taken to ensure that images labeled as having cactus present are accurate. While it would cause a waste of labor to move to intervene in an area that was incorrectly marked as not having cacti, it would be more problematic to miss impacted areas all together. This would mean areas that are in need of restoration would go unnoticed. 


In [1]:
#Import Relevant Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input, LSTM, AveragePooling2D
from keras.datasets import mnist
from keras import regularizers, initializers, optimizers
import os
import datetime
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input, LSTM
from keras.datasets import mnist
from keras import regularizers, initializers, optimizers
import os
import datetime
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path



from pathlib import Path

from sklearn.metrics import roc_auc_score, plot_confusion_matrix
import seaborn as sns

import gc
from timeit import default_timer as timer
import tensorflow as tf
pd.set_option('display.float_format', lambda x: '%.1f' % x)
os.environ['KMP_DUPLICATE_LIB_OK']='True' #This prevents kernel shut down due to xgboost conflict
from tensorflow.keras.preprocessing import image


import zipfile
import random

import cv2
from tensorflow import keras



from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Reshape, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pydot
import graphviz
from tensorflow.keras.utils import plot_model



print(os.listdir("../input/aerial-cactus-identification/"))

In [2]:

def plot_performance(hist):
    """ Returns 5 plots comparing Training and Validation data. 
    First plot returns training and validation accuracy. 
    Second plot returns training and validation loss. 
    Third plot returns training and validation F1-Scores. 
    Fourth plot returns training and validation recall scores. 
    
    hist: input history model containing train images, labels, and validation data. """
    
    hist_ = hist.history
    epochs = hist.epoch
    
    plt.plot(epochs, hist_['accuracy'], label='Training Accuracy')
    plt.plot(epochs, hist_['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, hist_['loss'], label='Training loss')
    plt.plot(epochs, hist_['val_loss'], label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    recall = np.array(hist_['recall'])
    precision = np.array(hist_['precision'])
    val_recall = np.array(hist_['val_recall'])
    val_precision = np.array(hist_['val_precision'])
    plt.figure()
    
    plt.plot(epochs, 
             2*((recall * precision)/(recall + precision)), 
             label='Training f1')
    plt.plot(epochs, 
             2*((val_recall * val_precision)/(val_recall + val_precision)), 
             label='Validation f1')
    plt.title('Training and validation F1-Score')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, recall, label = "Training Recall")
    plt.plot(epochs, val_recall, label = "Validation Recall")
    plt.title("Training and Validation Recall Scores")
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, precision, label = "Training Precision")
    plt.plot(epochs, val_precision, label = "Validation Precision")
    plt.title("Training and Validation Precision Scores")
    plt.legend()
    plt.figure()
    
    plt.show()
    


First, we exract our images from their zip files, and save them to a temporary directory for ease of working. We then can take a look at how many images are present in both our train and test files. 

In [3]:
# Extract images from zip files

with zipfile.ZipFile("../input/aerial-cactus-identification/train.zip","r") as z:
    z.extractall("/kaggle/temp/")
with zipfile.ZipFile("../input/aerial-cactus-identification/test.zip","r") as z:
    z.extractall("/kaggle/temp/test/") # needs to be in subdirectory (i.e. test/test/) for flow_from_directory to work


#Looking at the number of images in our train and test files
print(len(os.listdir("../temp/train")))
print(len(os.listdir("../temp/test/test")))

In [4]:
#Setting up our working directories
train_dir = "../temp/train"
test_dir = "../temp/test"
labels = pd.read_csv('../input/aerial-cactus-identification/train.csv')


# Image Analysis Overview

We can see from the count, and visualized in the graph below, the classes are very imbalanced. There are three times as many cacti images as there are no cacti images. We will later address this imbalance by augmenting the photos.

In [5]:
#Taking a look at the class balance 
labels.has_cactus = labels.has_cactus.astype(str)
print(labels['has_cactus'].value_counts())

In [6]:
sns.histplot(data = labels['has_cactus'], shrink = .8)

We can also take a look at a random sampling of the training images in our dataset. We can see that the images are not of the highest resolution which is to be expected from images taken from 100m, but the cacti are distinct.

In [7]:
# Plot random sample of training images

rand_images = random.sample(os.listdir(train_dir), 16)

fig = plt.figure(figsize=(16,4))
for i, im in enumerate(rand_images):
    plt.subplot(2, 8, i+1)
    im = cv2.imread(os.path.join(train_dir, im))
    plt.imshow(im)
    plt.axis('off')
plt.show()

## Performing a train test split

Here we split our data into our training set and our validation set. 80% of the images are going into the training set with the remaining images being used to validate our models performance.

In [8]:
#Splits the data randomly
val_split = 0.8
index = np.random.permutation(range(len(labels))) < val_split*len(labels)

train_labels = labels[index]
val_labels = labels[~index]
print(len(train_labels), len(val_labels))

## Image Data Pre-Processing

We want to start pre-processing our images before putting them through the models. Our pre-processing will ensure all images are the same size and are scaled down to relieve any computational pressure. In the initial image data generator, we flip our images horizontally and vertically to give our model more images from which to train. We then use flow from dataframe to utilize the augmentation from our image data generator.

As we set up our train and validation generators, we have to make sure to specify our batch size and target size. It is important as well to indicate that we are working on a binary classification problem by setting class_mode to 'binary'.

In [9]:
# Process image JPEGs into tensors
# Pixel values rescaled from [0,255] to [0,1]

# Generate batches of tensor image data (with real-time data augmentation - horizontal and vertical flips)
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255, horizontal_flip=True, vertical_flip=True)

batch_size = 128

train_generator = train_datagen.flow_from_dataframe(train_labels,directory=train_dir,x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(32,32))
val_generator = train_datagen.flow_from_dataframe(val_labels,directory=train_dir,x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(32,32))

We can now set our input shape that will be refelective of the target_size given above, and also the number of layers found in our images. Because we are working with RGB images, our layers are set to 3.

In [10]:
input_shape = (32, 32, 3)

# Baseline Model

In the first model, we would like to start simply to see what kind of results we can get with a little complication as possible. 

We start with an input layer to establish the structure of the incoming data. We then include a flattening layer to set our images into one long array. Two simple dense layers are then added which funnel the information down. The first dense layer uses a relu activation function. Our final dense layer uses a sigmoid activation function because we are working on a binary classification problem.

In [11]:
#First simple model
model_start = Sequential(
    [
        Input(input_shape),
        Flatten(), # need to flatten our images to be one long array
        Dense(64,activation="relu"),
        Dense(1, activation="sigmoid"),  
        
    ])

model_start.summary()

We now compile our model using binary crossentropy to evaluate our loss. We also use adam as our optimizer and evaluate using the accuracy, recall and precision metrics.. 

In [12]:
model_start.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy', 'Recall', 'Precision'])

We now can move to fitting our baseline model. We will set the number of epochs to 20 to get a quick understanding of our models performance. 

In [13]:
history = model_start.fit(train_generator, 
                     batch_size=batch_size,
                     epochs=20, 
                     validation_data=val_generator)

In [14]:
score1 = model_start.evaluate(val_generator, verbose=0)
print("Test loss:", score1[0])
print("Test accuracy:", score1[1])
print("Test recall:", score1[2])
print("Test precision:", score1[3])
plot_performance(history)


## Model Evaluation

Our first simple model has an overall test accuracy of 88%. Our test recall is 93%. Most importantly, our precision is 91%. This is a great start for our modeling. Precision is our most important metric because it is evaluating how often an image is accurately reporting that cacti are present. We need to be able to trust this metric because a mislabeling of a land as having cacti will lead to the land getting ignored. 

# Model Iterations

In model two, we wanted to play around with diferent types of layers to see their effects.

Our first layer is a Conv2D layer using the relu activation function. Relu is useful over tanh and sigmoid in the topmost layers because it makes the model easier to train and often the model achieves better performance. Setting 'same' to padding extends the area in which CNN processes an image. We also make sure to set our input shape. Our initial filter is set in this layer which determines the number of filters from which the CNN will learn.

We then include a MaxPooling2D layer. This operation is putting a 2D filter over each channel and summarising the features covered by the filter. This is performing downsampling on our photos, specifically on the height and width of the spatial dimensions.

We then flatten our matrix down to a single long array before putting it through two more Dense layers.

In compiling our model we are using an adam optimizer with a set learning rate of .01. Additionally, we have put in an early stop in this model. Early stop will make sure to stop the model learning at a point where it seems there are no more improvements to be made on model function.

We are giving the next models more epochs to run through to evaluate performance over more iterations. We can set the epoch to a high number, and trust that the early stop will end the model training when no improvements are being made. 

## Model 2

In [15]:
model2 = Sequential([

        Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape = input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation = 'relu'),
        Dense(1, activation='sigmoid')

]
)
opt = keras.optimizers.Adam(learning_rate=0.01)

# compiling models
model2.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy', 'Recall', 'Precision'])

# early stopping
cp = EarlyStopping(patience = 5, restore_best_weights=True)

model2.summary()

In [16]:
start = timer()

history2 = model2.fit(train_generator,
                    batch_size = batch_size,
                    epochs=50, 
                    validation_data = val_generator,
                    callbacks = [cp]
                    
                  
                   )
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [17]:
score2 = model2.evaluate(val_generator, verbose=0)
print("Test loss:", score2[0])
print("Test accuracy:", score2[1])
print("Test recall:", score2[2])
print("Test precision:", score2[3])
plot_performance(history2)

## Model 2 Evaluation 

Model 2 has an overall test accuracy of 96%. Our test recall is 97%. Most importantly, our precision is 97%. This is a massive improvement on our first model but we are seeing the recall scores jump all over the place between 91% and 98%. This could be a sign we need to let our models train for longer so in future iterations we will increase the number of epochs to 100. 

## Model 3

In model 3, we are keeping the same layers for a our model, but we are making some changes to the early stopping and adding in a feature that allows the learning rate to gradually adjust as needed. We are also going to allow the model to train for 100 epochs. 

In [18]:
model3 = Sequential([

        Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape = input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation = 'relu'),
        Dense(1, activation='sigmoid')

]
)

model3.summary()

In [19]:
model3.compile(loss = keras.losses.binary_crossentropy,
              optimizer = 'adam',
              metrics = ['accuracy', 'Recall', 'Precision'])
#Very cool early stopping and learning rate work
cp = [EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True), # Stop training when a monitored metric has stopped improving
             ReduceLROnPlateau(patience=10, verbose=1), # Reduce learning rate when a metric has stopped improving

            ]

In [20]:
history3 = model3.fit(train_generator,
                    epochs = 100,
                    verbose = 1,
                    callbacks = cp,
                    validation_data = val_generator,
                    
                   )

In [21]:
score3 = model3.evaluate(val_generator, verbose=0)
print("Test loss:", score3[0])
print("Test accuracy:", score3[1])
print("Test recall:", score3[2])
print("Test precision:", score3[3])
plot_performance(history3)



In [22]:


model6 = Sequential([

    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
    
])

model6.summary()

In [23]:
model6.compile(loss = keras.losses.binary_crossentropy,
              optimizer = 'adam',
              metrics = ['accuracy', 'Recall', 'Precision'])

cp = [EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True), # Stop training when a monitored metric has stopped improving
             ReduceLROnPlateau(patience=10, verbose=1), # Reduce learning rate when a metric has stopped improving

            ]

In [24]:
history6 = model6.fit(train_generator,
                    epochs = 100,
                    verbose = 1,
                    callbacks = cp,
                    validation_data = val_generator,
                    #class_weight = class_weights,
                   )

In [25]:
score6 = model6.evaluate(val_generator, verbose=0)
print("Test loss:", score6[0])
print("Test accuracy:", score6[1])
print("Test recall:", score6[2])
print("Test precision:", score6[3])

plot_performance(history6)

In [26]:
preds = model6.predict(val_generator, verbose = 1)

In [27]:
test = val_generator

test.reset()
x=np.concatenate([test.next()[0] for i in range(test.__len__())])
y=np.concatenate([test.next()[1] for i in range(test.__len__())])
print(x.shape)
print(y.shape)

In [28]:
plot_model(model6,show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

dic = {0:'No', 1:'Yes'}
plt.figure(figsize=(20,20))
for i in range(0+228, 16+228):
    
    plt.subplot(4, 4, (i-228)+1)
    if preds[i, 0] >= 0.5: 
        out = ('{:.2%} probability of HAVING cacti'.format(preds[i][0]))
    else: 
        out = ('{:.2%} probability of NOT HAVING cacti'.format(1-preds[i][0]))
    plt.title(out+"\n Cacti : "+ dic.get(y[i]))    
    plt.imshow(np.squeeze(x[i]))
plt.axis('off')
plt.show()

# Conclusion